In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GRQO_API_KEY")
llm = ChatGroq(groq_api_key= groq_api_key, model="Llamma3.2-8b-8192")

